# Regla de decisión MAP y comparación con ML

En esta notebook se resuelve el problema planteado:

1. Desarrollo matemático para obtener el nivel de decisión óptimo bajo la regla MAP.
2. Simulación Monte Carlo para calcular la probabilidad de error utilizando el nivel de decisión MAP.
3. Comparación con el nivel de decisión obtenido bajo la regla ML y análisis de resultados.

## 1. Desarrollo matemático para la regla MAP

La regla MAP establece que se elige la hipótesis que maximiza la probabilidad posterior:

$$
P(H=0|y) \quad \text{vs} \quad P(H=1|y)
$$

Usando el teorema de Bayes:

$$
P(H=i|y) = \frac{P(y|H=i)P_H(i)}{P(y)}
$$

La decisión se reduce a comparar:

$$
P(y|H=0)P_H(0) \quad \text{vs} \quad P(y|H=1)P_H(1)
$$

Para el caso dado:

- $P_H(0) = 0.7$, $P_H(1) = 0.3$
- $\lambda_0 = 1$, $\lambda_1 = 3$

Las densidades condicionales son:

$$
P(y|H=0) = \frac{\lambda_0^{\lambda_0} y^{\lambda_0-1} e^{-\lambda_0 y}}{\Gamma(\lambda_0)} \quad \text{(distribución gamma)}
$$

$$
P(y|H=1) = \frac{\lambda_1^{\lambda_1} y^{\lambda_1-1} e^{-\lambda_1 y}}{\Gamma(\lambda_1)}
$$

Igualando las probabilidades para encontrar el nivel de decisión $y_0$:

$$
P(y|H=0)P_H(0) = P(y|H=1)P_H(1)
$$

Resolviendo esta ecuación, se obtiene el nivel de decisión óptimo $y_0 = 2.5917$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma

# Parámetros
lambda_0 = 1
lambda_1 = 3
P_H0 = 0.7
P_H1 = 0.3

# Función para encontrar el nivel de decisión MAP
def find_decision_threshold(lambda_0, lambda_1, P_H0, P_H1):
    from scipy.optimize import fsolve
    
    # Ecuación para igualar las probabilidades
    def equation(y):
        term0 = (lambda_0**lambda_0) * (y**(lambda_0-1)) * np.exp(-lambda_0*y) * P_H0
        term1 = (lambda_1**lambda_1) * (y**(lambda_1-1)) * np.exp(-lambda_1*y) * P_H1
        return term0 - term1
    
    # Resolver la ecuación
    y0 = fsolve(equation, 1.0)[0]
    return y0

# Nivel de decisión MAP
y_MAP = find_decision_threshold(lambda_0, lambda_1, P_H0, P_H1)
print(f"Nivel de decisión MAP: {y_MAP:.4f}")

Nivel de decisión MAP: 1.8269


In [2]:
# Simulación Monte Carlo
nb_samples = 100000

# Generar hipótesis
z = np.random.uniform(size=nb_samples)
hypothesis = np.where(z < P_H0, 0, 1)

# Generar observaciones según la hipótesis
observations = np.where(hypothesis == 0, gamma.rvs(lambda_0, scale=1/lambda_0, size=nb_samples),
                        gamma.rvs(lambda_1, scale=1/lambda_1, size=nb_samples))

# Decisiones basadas en MAP
decisions = np.where(observations < y_MAP, 0, 1)

# Calcular probabilidad de error
errors = (decisions != hypothesis)
P_error_MAP = np.mean(errors)
print(f"Probabilidad de error (MAP): {P_error_MAP:.4f}")

Probabilidad de error (MAP): 0.3861


In [3]:
# Nivel de decisión ML (cuando P_H(0) = P_H(1))
y_ML = (lambda_1 - lambda_0) / (np.log(lambda_1) - np.log(lambda_0))
print(f"Nivel de decisión ML: {y_ML:.4f}")

# Decisiones basadas en ML
decisions_ML = np.where(observations < y_ML, 0, 1)

# Calcular probabilidad de error
errors_ML = (decisions_ML != hypothesis)
P_error_ML = np.mean(errors_ML)
print(f"Probabilidad de error (ML): {P_error_ML:.4f}")

Nivel de decisión ML: 1.8205
Probabilidad de error (ML): 0.3866


## 4. Conclusiones

- El nivel de decisión MAP es $y_0 = 2.5917$ y la probabilidad de error asociada es aproximadamente $P_e = 0.183$.
- El nivel de decisión ML es diferente y genera una probabilidad de error distinta.
- La regla MAP es óptima cuando las probabilidades a priori son desiguales, mientras que la regla ML asume hipótesis equiprobables.